In [ ]:
"""
Use this code to save the result to a csv file
"""



import requests
import json
import csv
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# Update scope to access Google Cloud services
SCOPES = ['https://www.googleapis.com/auth/cloud-platform']

def get_credentials():
    creds = None
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    return creds

def search_nearby(creds, southwest_lat, southwest_lng, northeast_lat, northeast_lng, included_types=None, max_result_count=20):
    base_url = 'https://places.googleapis.com/v1/places:searchNearby'
    
    # Construct the JSON payload
    payload = {
        "locationRestriction": {
            "rectangle": {
                "southwest": {
                    "latitude": southwest_lat,
                    "longitude": southwest_lng
                },
                "northeast": {
                    "latitude": northeast_lat,
                    "longitude": northeast_lng
                }
            }
        },
        "maxResultCount": max_result_count
    }
    
    if included_types:
        payload["includedTypes"] = included_types
    
    # Define headers
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {creds.token}',
        'X-Goog-FieldMask': 'places.displayName'
    }
    
    # Send POST request
    response = requests.post(base_url, headers=headers, data=json.dumps(payload))
    
    # Check for successful response
    if response.status_code == 200:
        places = response.json()
        return places
    else:
        print(f"Error: {response.status_code}")
        print(response.json())
        return None


def save_to_csv(places, filename='places.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['DisplayName', 'Types', 'PriceLevel', 'Rating'])
        
        for place in places:
            writer.writerow([
                place.get('displayName', {}).get('text', 'No name available'),
                ','.join(place.get('types', [])),
                place.get('priceLevel', 'No price level available'),
                place.get('rating', 'No rating available')
            ])

# Example usage
if __name__ == "__main__":
    southwest_lat = -8.732
    southwest_lng = 115.160
    northeast_lat = -8.727
    northeast_lng = 115.170
   
    
    creds = get_credentials()
    places = search_nearby(creds, southwest_lat, southwest_lng, northeast_lat, northeast_lng)

    if places:
        save_to_csv(places)
        print(f"Saved {len(places)} places to places.csv")
        
        # for place in places.get('places', []):  # Adjust to access 'places' key instead of 'results'
        #     # print(place['displayName']['text'])  # Access 'text' within 'displayName' dictionary
        #     print(place.get('displayName', 'No name available'))
        #     print(place.get('types', []))  # Access 'types' key and return empty list if not found
        #     print(place.get('priceLevel', 'No price level available'))
        #     print(place.get('rating', 'No rating available'))
        #     print()
